In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

## Resilient distributed systems

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 10:46:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

## filter out old data

In [6]:
rows = rdd.take(10)

In [8]:
row = rows[0]

In [9]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [10]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [11]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [12]:
from collections import namedtuple

In [13]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [14]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )


In [16]:
rdd\
    .filter(filter_outliers)\
    .map(prepare_for_grouping)\
    .reduceByKey(calculate_revenue)\
    .map(unwrap)\
    .toDF()\
    .show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [17]:
from pyspark.sql import types

In [18]:

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [19]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [20]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [23]:
df_result.write.parquet('tmp/green-revenue', mode = 'overwrite')

In [ ]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd